In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
print(cv.__version__)

4.2.0


In [2]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")
V = ry.ConfigurationViewer()
V.setConfiguration(RealWorld)

In [3]:
#change some colors
RealWorld.getFrame("obj0").setColor([0,1,0])
RealWorld.getFrame("obj1").setColor([1,0,0])
RealWorld.getFrame("obj2").setColor([1,1,0])
RealWorld.getFrame("obj3").setColor([1,0,1])
RealWorld.getFrame("obj4").setColor([0,1,1])

#you can also change the shape & size
RealWorld.getFrame("obj0").setColor([1.,0,0])
RealWorld.getFrame("obj0").setShape(ry.ST.sphere, [.03])
#RealWorld.getFrame("obj0").setShape(ry.ST.ssBox, [.05, .05, .2, .01])
RealWorld.getFrame("obj0").setPosition([0., .2, 2.])

#remove some objects
for o in range(5,30):
    name = "obj%i" % o
    print("deleting", name)
    RealWorld.delFrame(name)

    
V.recopyMeshes(RealWorld)
V.setConfiguration(RealWorld)

deleting obj5
deleting obj6
deleting obj7
deleting obj8
deleting obj9
deleting obj10
deleting obj11
deleting obj12
deleting obj13
deleting obj14
deleting obj15
deleting obj16
deleting obj17
deleting obj18
deleting obj19
deleting obj20
deleting obj21
deleting obj22
deleting obj23
deleting obj24
deleting obj25
deleting obj26
deleting obj27
deleting obj28
deleting obj29


In [4]:
# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [5]:
# we're adding an "imp" to the simulation, which is a little process that can inject perturbations
S.addImp(ry.ImpType.objectImpulses, ['obj0'], [])

In [6]:
# create your model world
C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
#V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")

#the focal length
f = 0.895
f = f * 360.
fxfypxpy = [f, f, 320., 180.]

In [7]:
def segment_ball(bgr):
    
    #if len(rgb)>0: cv.imshow('OPENCV - rgb', bgr)

    hsv = cv.cvtColor(bgr, cv.COLOR_BGR2HSV)
    lower_red = np.array([0,120,70])
    upper_red = np.array([10,255,255])

    mask1 = cv.inRange(hsv, lower_red, upper_red)

    # second red component
    lower_red = np.array([170,120,70])
    upper_red = np.array([180,255,255])
    mask2 = cv.inRange(hsv,lower_red,upper_red)


    # combine mask ( + does or operation) to generate binary image with white as red color
    mask = mask1 + mask2
    if len(mask)>0: cv.imshow('OPENCV - binary', mask)

    blur_mask = cv.GaussianBlur(mask,(9,9),3,3)

#     weighted_mask = cv.addWeighted(mask1, 1.0, mask2, 1.0, 0.0)

#     # then the result is blurred
#     blurred_mask = cv.GaussianBlur(weighted_mask,(9,9),3,3)

#     # some morphological operations (closing) to remove small blobs 
#     erode_element = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))
#     dilate_element = cv.getStructuringElement(cv.MORPH_RECT, (8, 8))
#     eroded_mask = cv.erode(blurred_mask,erode_element)
#     dilated_mask = cv.dilate(eroded_mask,dilate_element)

    circles = cv.HoughCircles(blur_mask, cv.HOUGH_GRADIENT, 1, 150, param1=100, param2=20, minRadius=0, maxRadius=13)
    black_img = np.zeros(bgr.shape)
    if circles is not None:
        for circle in circles[0, :]:
            circled_orig = cv.circle(black_img, (circle[0], circle[1]), circle[2], (255,255,255),thickness=-1)

        #circled_orig_grey = cv.cvtColor(circled_orig, cv.COLOR_BGR2GRAY)
        circled_orig_blur = cv.GaussianBlur(circled_orig[:,:,0],(3,3),3,3)
        cv.imshow("circle_mask", circled_orig_blur)
        return circled_orig_blur

    return None

In [10]:
points = []
tau = .01

for t in range(300):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
        #cameraFrame.setPointCloud(points, rgb)
        bgr = cv.cvtColor(rgb, cv.COLOR_RGB2BGR)
        circled_orig_blur = segment_ball(bgr)
        
        pc_coord = []
        if circled_orig_blur is not None:
            x,y = np.where(circled_orig_blur)
            full_depth = np.zeros(points.shape)
            for i, j in zip(x,y):
                full_depth[i][j] = points[i][j] 
                pc_coord.append(points[i][j])
            pc_coord =(np.array(pc_coord))
            mean_pc = np.mean(pc_coord, axis=0)
            cameraFrame.setPointCloud(full_depth, rgb)
                    
        V.recopyMeshes(C)
        V.setConfiguration(C)
        
        #if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        
    S.step([], tau, ry.ControlMode.none)

In [11]:
cv.destroyAllWindows()


In [12]:
percept = C.addFrame("percept", "camera")
percept.setQuaternion([1,0,0,0])
percept.setShape(ry.ST.sphere, [.03])
percept.setColor([0,1,0])
percept.setPosition(cameraFrame.getRotationMatrix() @ mean_pc + cameraFrame.getPosition())
V.setConfiguration(C)


In [13]:
# C.addObject(name='item7', parent='camera', shape=ry.ST.sphere, pos=list(mean_pc), size=[.5], color=[0., 1., 0.])
# V.setConfiguration(C)

In [14]:
percept.setContact(1)
R_gripper = C.frame("R_gripper")
L_gripper = C.frame("L_gripper")
R_gripper.setContact(1)
#L_gripper.setContact(1)


In [15]:
komo = C.komo_path(1.,20, 5., True) 
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])

komo.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "percept"], ry.OT.sos, [1e3])
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.distance, ["R_gripperCenter", "percept"], ry.OT.eq, [1e2]);

# komo.addObjective([1.], ry.FS.scalarProductXZ, ["percept", "R_gripperCenter"], ry.OT.eq)
# komo.addObjective([1.], ry.FS.scalarProductXZ, ["R_gripperCenter", "percept"], ry.OT.eq)

komo.addObjective(time=[1.], feature=ry.FS.qItself, type=ry.OT.eq, order=1);
komo.optimize(True)
komo.getReport()

[{'x_dim': 320, 'T': 20, 'k_order': 2, 'tau': 0.25, 'useSwift': True},
 {'order': 2.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 247.5790848212679},
 {'order': 0.0,
  'type': 'ineq',
  'feature': 'ProxyCost',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'inEq_sumOfPos': 25.772480805987566},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'Default-0-posDiff-R_gripperCenter-percept',
  'vars': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  'sos_sumOfSqr': 84.43534612858925},
 {'order': 0.0,
  'type': 'ineq',
  'feature': 'qLimits',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'inEq_sumOfPos': 28.362629336449032},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'PairCollision-R_gripperCenter-percept',
  'vars': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  'eq_sumOfAbs': 2.081112922451353},
 {'order': 1.0,
  'ty

In [16]:
kv = komo.view()

In [17]:
kv.playVideo()

In [18]:
kv=0

In [19]:
C.setFrameState( komo.getConfiguration(19) )
V.setConfiguration(C)
C.getJointState()

array([ 1.68602979e-02, -1.25770289e+00, -6.18099014e-01, -1.89076177e+00,
        7.59635006e-01,  2.72367080e+00,  2.49007852e-01,  1.31383098e-02,
       -3.04560161e+00, -1.01230637e+00, -2.57635439e+00, -1.95206375e+00,
       -2.27024230e-01, -3.05268105e+00, -2.60052888e-03,  1.02334038e-02])

In [20]:
Xstart = C.getFrameState()
S.setState(Xstart)

In [21]:
q = S.get_q()
tau = .01
S.step(q, tau, ry.ControlMode.position)

In [23]:
for i in range(1000):
    time.sleep(0.01)
    q = S.get_q()
    S.closeGripper("R_gripper")
    S.step(q, tau, ry.ControlMode.position)
    

In [20]:
C.attach("R_gripper", "percept")

In [ ]:
#move a bit around

q = C.getJointState()

for t in range(30):
    q[0] = np.sin(t/10)
    
    C.setJointState(q)
    V.setConfiguration(C)
    time.sleep(0.1)

In [26]:
S.getGripperIsGrasping("R_gripper")

False

In [36]:
S.closeGripper("R_gripper")
S.steap

In [31]:
S.openGripper("R_gripper")

In [37]:
q = S.get_q()
S.openGripper("R_gripper")
S.step(q, tau, ry.ControlMode.position)

In [ ]:
c=0
V=0
S=0